In [13]:
# Import statements 
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import pandas as pd
import string
import datetime
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

In [14]:
# Load data
train_feats = pd.read_csv('train-dev-test/train_cleaned_features.csv', delimiter=',', encoding="utf-8")
dev_feats = pd.read_csv('train-dev-test/dev_cleaned_features.csv', delimiter=',', encoding="utf-8")
test_feats = pd.read_csv('train-dev-test/test_cleaned_features.csv', delimiter=',', encoding="utf-8")

# Clear whitespace
train_feats.columns = train_feats.columns.str.strip()

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (3,4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [15]:
# Dropping stuff!

# Drop all rows where time is NA?
print("Old length: ", len(train_feats))
train_feats = train_feats[train_feats['time'].notna()]
print("New length: ", len(train_feats))

# Just drop hashtags for now, too difficult for M3 to deal with
train_feats = train_feats.drop(['hashtags'], axis=1)

Old length:  3515562
New length:  3515538


In [16]:
# View the dataframes
train_feats.head(5)
#train_feats.columns                                        

,text,place,user_location,followers_count,retweet_count,favorite_count,weekday,month,day,time,...,num_present,trump_present,hashtag_present,covid,vaccine,profanity_present,emoji_present,url_present,question_exclamation_present,sentiment
0,Pretty much any corona virus germ that wants t...,NaN,PG County,478,0,10,Thu,Mar,19.0,19:52:15,...,False,False,False,True,False,False,False,True,False,positive
1,RT @RealSaavedra: Good.\n\nIt came from China.,NaN,"Sydney, New South Wales",394,102,0,Thu,Mar,19.0,19:52:15,...,False,False,False,False,False,False,False,False,False,positive
2,RT @nicolebyer: Everyone in this looks sick,NaN,ferrets,695,48499,0,Thu,Mar,19.0,19:52:15,...,False,False,False,False,False,False,False,False,False,negative
3,RT @jabinbotsford: Close up of President @real...,NaN,NaN,19,30492,0,Thu,Mar,19.0,19:52:15,...,False,True,False,True,False,False,False,False,False,neutral
4,RT @Biancaixvi: Corona day 3: it just feels li...,NaN,"Huntley, IL",392,85302,0,Thu,Mar,19.0,19:52:15,...,True,False,False,True,False,False,False,False,False,neutral


In [17]:
train_feats.dtypes

text                             object
place                            object
user_location                    object
followers_count                  object
retweet_count                    object
favorite_count                   object
weekday                          object
month                            object
day                             float64
time                             object
year                            float64
num_present                        bool
trump_present                      bool
hashtag_present                    bool
covid                              bool
vaccine                            bool
profanity_present                  bool
emoji_present                      bool
url_present                        bool
question_exclamation_present       bool
sentiment                        object
dtype: object

In [18]:
# Feature manipulation to make them ready for the model
# Change sentiment labels to numbers!
train_feats['sentiment'] = train_feats['sentiment'].astype('category')
train_feats["sentiment"] = train_feats["sentiment"].cat.codes

# Change location
train_feats["user_location"] = train_feats["user_location"].astype('category')
train_feats["user_location"] = train_feats["user_location"].cat.codes

# Change place
train_feats["place"] = train_feats["place"].astype('category')
train_feats["place"] = train_feats["place"].cat.codes

# Change weekday
#train_feats["weekday"] = train_feats["weekday"].astype('category')
#train_feats["weekday"] = train_feats["weekday"].cat.codes

# Change month
train_feats["month"] = train_feats["month"].astype('category')
train_feats["month"] = train_feats["month"].cat.codes

# Change time to datetime object
#train_feats['time'] = train_feats['time'].apply(lambda x: datetime.datetime.strptime(str(x), '%H:%M:%S'))

# Add hour and minute columns
#train_feats['hour'] = train_feats['time'].apply(lambda x: x.hour)
#train_feats['minute'] = train_feats['time'].apply(lambda x: x.minute)

# Drop time
train_feats = train_feats.drop(['time'], axis=1)

# Change all the bool types to numeric
train_feats["num_present"] = train_feats["num_present"].astype(int)
train_feats["trump_present"] = train_feats["trump_present"].astype(int)
#train_feats["hashtag_present"] = train_feats["hashtag_present"].astype(int)
train_feats["covid"] = train_feats["covid"].astype(int)
train_feats["vaccine"] = train_feats["vaccine"].astype(int)
train_feats["profanity_present"] = train_feats["profanity_present"].astype(int)
#train_feats["emoji_present"] = train_feats["emoji_present"].astype(int)
train_feats["url_present"] = train_feats["url_present"].astype(int)
#train_feats["question_exclamation_present"] = train_feats["question_exclamation_present"].astype(int)

In [19]:
all_text = train_feats['text'].tolist()

vectorizer = TfidfVectorizer(max_features=100, min_df=7, max_df=0.8, stop_words=stopwords.words('english'))
processed_features = vectorizer.fit_transform(all_text).toarray()

In [20]:
# Drop text (JUST FOR A TEST)
train_feats = train_feats.drop(['text'], axis=1)
train_feats = train_feats.drop(['weekday'], axis=1)

In [29]:
# Get data ready
# Get sentiment
labels = train_feats['sentiment']
# Drop labels from larger dataframee
train_feats = train_feats.drop(['sentiment'], axis=1)

# Convert both to numpy arrays
#non_text_feats = train_feats.to_numpy()
#X = np.concatenate((processed_features, non_text_feats), axis=1)
X = processed_features
y = labels.to_numpy()

KeyError: 'sentiment'

In [30]:
# Instantiate model - Multinomial logistic regression
model = LogisticRegression(multi_class='multinomial', solver='lbfgs')

# Train model
model.fit(X, y)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(multi_class='multinomial')

In [31]:
y_preds = model.predict(X)
# Print score
print(accuracy_score(y_preds, y))

0.6274510473219177


In [32]:
from sklearn.metrics import f1_score
print(f1_score(y, y_preds, average='micro'))

0.6274510473219177
